In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
tit_train=pd.read_csv('/kaggle/input/titanic/train.csv')

In [ ]:
tit_train.head(10)

In [ ]:
tit_train.info()

In [ ]:
tit_train.describe()

In [ ]:
tit_test=pd.read_csv('/kaggle/input/titanic/test.csv')

In [ ]:
tit_test.head(10)

In [ ]:
tit_test.info()

In [ ]:
tit_test.describe()

In [ ]:
tit_train.isnull().sum()

In [ ]:
sns.countplot(x='Survived',data=tit_train, color='green', saturation=0.1)
plt.show()


In [ ]:
sns.countplot(x='Pclass',data=tit_train)

In [ ]:
sns.countplot(x='Age',data=tit_train)

In [ ]:
sns.countplot(x='SibSp',data=tit_train)

In [ ]:
sns.countplot(x='Parch',data=tit_train)

In [ ]:
sns.countplot(x='Embarked',data=tit_train)

In [ ]:
sns.displot(tit_train['Age'])

In [ ]:
grid = sns.FacetGrid(tit_train, col='Survived', row='Pclass')
grid.map(plt.hist, 'Age', alpha=.5, bins=20)
grid.add_legend();

In [ ]:
fig,axes = plt.subplots(2,2,figsize=(12,8))
sns.histplot(x="Age",data=tit_train,ax=axes[0,0])
sns.countplot(x="SibSp",data=tit_train,ax=axes[0,1],hue='Survived')
sns.boxplot(x="Pclass",y='Age',data=tit_train,ax=axes[1,0])
sns.pointplot(x="Survived",y='Fare',data=tit_train,ax=axes[1,1],estimator='median')

In [ ]:
sns.scatterplot(x='Age',y='Fare',data=tit_train,hue='Survived')

In [ ]:
corr = tit_train.corr()
sns.heatmap(corr,annot=True)

In [ ]:
tit_train['Age'] = tit_train['Age'].fillna(tit_train['Age'].mean())

In [ ]:
tit_train.isnull().sum()

In [ ]:
tit_test.isnull().sum()

In [ ]:
tit_test['Fare'] = tit_test['Fare'].fillna(tit_test['Fare'].mean())
tit_test['Age'] = tit_test['Age'].fillna(tit_test['Age'].mean())

In [ ]:
from sklearn.model_selection import train_test_split

y = tit_train["Survived"]

dummies = pd.get_dummies(tit_train,columns=['Sex','Embarked'],drop_first=True)
X = dummies.drop(['Survived','Name','Ticket','PassengerId','Cabin'],axis=1)

x_train,x_test,y_train,y_test = train_test_split(X,y)

In [ ]:
dummies = pd.get_dummies(tit_test,columns=['Sex','Embarked'],drop_first=True)
X_test = dummies.drop(['Name','Ticket','PassengerId','Cabin'],axis=1)
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(criterion='gini', n_estimators=700,
                             min_samples_split=10,min_samples_leaf=1,
                             max_features='auto',oob_score=True,
                             random_state=1,n_jobs=-1)
model.fit(x_train, y_train)
predictions = model.predict(x_test)
model.score(x_test, y_test)

In [ ]:
X_test.isnull().sum()

In [ ]:
Y_pred = model.predict(X_test)

In [ ]:
output = pd.DataFrame({'PassengerId':tit_test['PassengerId'],'Survived':Y_pred})
output = output.to_csv('submission_tatanic.csv',index=False)

In [ ]:
def tunning_performance(classifier, model_name):
    print(model_name)
    print('Best Score: ' + str(classifier.best_score_))
    print('Best Parameters: ' + str(classifier.best_params_))

In [ ]:
from sklearn.model_selection import GridSearchCV 
rm = RandomForestClassifier(random_state = 1)
param_grid =  {'n_estimators': [300,350,400,450],
               'criterion':['gini','entropy'],
                                  'bootstrap': [True],
                                  'max_depth': [10,21,20],
                                  'max_features': ['auto','sqrt', 10],
                                  'min_samples_leaf': [4,5],
                                  'min_samples_split': [4,5]}
                                  
clf_rf = GridSearchCV(rm, param_grid = param_grid, cv = 9, verbose = True, n_jobs = -1)
best_clf_rf = clf_rf.fit(x_train,y_train)
tunning_performance(best_clf_rf,'Random Forest')